<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/Pending/slater_rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/andrejewski/periodic-table
import pandas as pd

df = pd.read_csv("electron.csv")
df

,atomicNumber,symbol,electronicConfiguration
0,1,H,1s1
1,2,He,1s2
2,3,Li,1s2 2s1
3,4,Be,1s2 2s2
4,5,B,1s2 2s2 2p1
...,...,...,...
113,114,Fl,6s2 6p6 5f14 6d10 7s2 7p2
114,115,Mc,6s2 6p6 5f14 6d10 7s2 7p3
115,116,Lv,6s2 6p6 5f14 6d10 7s2 7p4
116,117,Ts,6s2 6p6 5f14 6d10 7s2 7p5


In [ ]:
def electron_config(s):
    """
    文字列を電子配置に変換する。

    s: csvファイルのelectronicConfigurationの内容
    """
    return {ec[:2]: int(ec[2:]) for ec in s.split()}


electron_config("6s2 6p6 5f14 6d10 7s2 7p2")

{'6s': 2, '6p': 6, '5f': 14, '6d': 10, '7s': 2, '7p': 2}

In [ ]:
full_occupancy = [["1s",2],
                  ["2s",2], ["2p",6],
                  ["3s",2], ["3p",6],
                  ["4s",2], ["3d",10], ["4p",6],
                  ["5s",2], ["4d",10], ["5p",6]]


def electron_config_completed(s):
    """
    内殻軌道で省略されているものを補う。
    """
    conf = electron_config(s)
    for orbital, nelec in full_occupancy:
        if orbital in conf:
            break
        conf[orbital] = nelec
    return conf

electron_config_completed("6s2 6p6 5f14 6d10 7s2 7p2")

{'6s': 2,
 '6p': 6,
 '5f': 14,
 '6d': 10,
 '7s': 2,
 '7p': 2,
 '1s': 2,
 '2s': 2,
 '2p': 6,
 '3s': 2,
 '3p': 6,
 '4s': 2,
 '3d': 10,
 '4p': 6,
 '5s': 2,
 '4d': 10,
 '5p': 6}

In [ ]:
# Slater
slater_groups = [{"1s"},
                 {"2s", "2p"},
                 {"3s", "3p"},
                 {"3d"},
                 {"4s", "4p"},
                 {"4d"}]
# これより外の軌道のことは知らない。


In [ ]:
def shielders(orbital, econf):
    """
    Slaterの考え方に基き、軌道orbitalの電子を遮蔽する、グループごとの電子数を数える。
    """
    enum = [0]*6
    for i, group in enumerate(slater_groups):
        for orbi in group:
            if orbi in econf:
                enum[i] += econf[orbi]
        if orbital in group:
            # orbitalの電子は1つ減らしておく。
            enum[i] -= 1
            enum = enum[:i+1]
            break
    return enum


def outermost_orbital(econf):
    """
    最外殻軌道を返す。
    """
    # highest occupied orbital group
    hoog = 0
    outermost = ""
    for i, group in enumerate(slater_groups):
        for orbi in group:
            if orbi in econf:
                if hoog < i:
                    hoog = i
                    outermost = orbi
    return outermost

econf = electron_config_completed("3p6")
econf

{'3p': 6, '1s': 2, '2s': 2, '2p': 6, '3s': 2}

In [ ]:
outermost = outermost_orbital(econf)
outermost

'3p'

In [ ]:
shielders(outermost, econf), 

([2, 7],)

In [ ]:
def slater_shielding(orbital, econf):
    """
    遮蔽定数の推定
    """
    sh = shielders(orbital, econf)
    if len(sh) == 1:
        # 内側の軌道がない==1s軌道のみがしゃへいする場合
        return 0.3
    elif len(sh) == 2:
        return sh[0]*0.85 + sh[1]*0.35
    # 0.80になる場合もあったと思うが、目をつぶる
    return sh[-1]*0.35 + sh[-2]*0.85 + sum(sh[:-2])


# K
econf = electron_config_completed("4s1")
outermost = outermost_orbital(econf)
slater_shielding(outermost, econf)

    

18.0